### Imports 

In [99]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import os

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Suppress warnings
warnings.filterwarnings('ignore')

# Feature Engineering

In [100]:
retail_df = pd.read_csv("/Users/darylwanji/Desktop/Data Projects /Darylwanji-Data-Insights-for-Call-Center-Efficiency-and-Performance/Data/CleanedRetail.csv")
retail_df

,CustomerID,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,TotalRevenue,Country
0,17850,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,15.30,United Kingdom
1,17850,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,20.34,United Kingdom
2,17850,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,22.00,United Kingdom
3,17850,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,20.34,United Kingdom
4,17850,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,20.34,United Kingdom
...,...,...,...,...,...,...,...,...,...
396158,12680,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,10.20,France
396159,12680,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12.60,France
396160,12680,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,16.60,France
396161,12680,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,16.60,France


In [101]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396163 entries, 0 to 396162
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CustomerID    396163 non-null  int64  
 1   InvoiceNo     396163 non-null  int64  
 2   StockCode     396163 non-null  object 
 3   Description   396163 non-null  object 
 4   Quantity      396163 non-null  int64  
 5   InvoiceDate   396163 non-null  object 
 6   UnitPrice     396163 non-null  float64
 7   TotalRevenue  396163 non-null  float64
 8   Country       396163 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 27.2+ MB


In [102]:
retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'])

In [103]:
print(f" Null values: \n {retail_df.isnull().sum()}") #check for null values

 Null values: 
 CustomerID      0
InvoiceNo       0
StockCode       0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
TotalRevenue    0
Country         0
dtype: int64


In [104]:
# check duplicateds
print(f" Total number of duplicates : {retail_df.duplicated().sum()}")

retail_df.drop_duplicates(inplace=True)

print(f" Total number of duplicates : {retail_df.duplicated().sum()}")


 Total number of duplicates : 5187
 Total number of duplicates : 0


In [105]:
retail_df['Year'] = retail_df['InvoiceDate'].dt.year
retail_df['Month'] = retail_df['InvoiceDate'].dt.month
retail_df['Day'] = retail_df['InvoiceDate'].dt.day
retail_df['Hour'] = retail_df['InvoiceDate'].dt.hour
retail_df['IsWeekend'] = retail_df['InvoiceDate'].dt.weekday >= 5

In [106]:
customer_summary = retail_df.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'TotalRevenue': 'sum'
}).rename(columns={'Quantity': 'TotalQuantity', 'TotalRevenue': 'TotalRevenuePerCustomer'})

In [107]:
latest_date = retail_df['InvoiceDate'].max()
rfm = retail_df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalRevenue': 'sum'
}).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalRevenue': 'MonetaryValue'})

In [108]:
rfm

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346,325,1,77183.60
12347,1,7,4310.00
12348,74,4,1437.24
12349,18,1,1457.55
12350,309,1,294.40
...,...,...,...
18280,277,1,180.60
18281,180,1,80.82
18282,7,2,178.05


# Base Model 

# K-Means Clustering 

# DBSCAN 